In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from medcat.cat import CAT
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

C:\Users\BILAL\AppData\Roaming\Python\Python37\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
!pip install nltk


     ---------------------------------------- 1.5/1.5 MB 8.7 MB/s eta 0:00:00


In [2]:
# Load model pack and Create CAT - the main class from medcat used for concept annotation

model_pack_path = 'medcat/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5'
cat = CAT.load_model_pack(model_pack_path)

Found an existing unziped model pack at: medcat\mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5, the provided zip will not be touched.
{
  "Model ID": "25be3857ba34bdd5",
  "Last Modifed On": "16 March 2022",
  "History (from least to most recent)": [
    "a474096eb4566638",
    "009617d7ff372682"
  ],
  "Description": "SNOMED INT enriched with UMLS and trained unsupervised on MIMIC-III",
  "Source Ontology": "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z",
  "Location": "MedCAT Rosalind machine, available for public download from https://github.com/CogStack/MedCAT",
  "MetaCAT models": {
    "Status": "Detects is a concept Affirmed or Negated/Hypothetical"
  },
  "Basic CDB Stats": {
    "Number of concepts": 354448,
    "Number of names": 2049216,
    "Number of concepts that received training": 29674,
    "Number of seen training examples in total": 20585988,
    "Average training examples per concept": 693.7382220125362
  },
  "Performance": {
    "ner": {},
    "meta"

In [3]:
test_notes = pd.read_csv('1000_testing_notes_with_smallest_size.csv')
test_notes

,Unnamed: 0,hadm_id,text,text_length
0,0,20019282,\nName: ___ Unit No: ___\n...,2480
1,1,20022343,\nName: ___ Unit No: _...,3001
2,2,20025548,\nName: ___ Unit No: ___\n...,2961
3,3,20027416,\nName: ___ Unit No: ___\n...,1059
4,4,20028418,\nName: ___ Unit No: ___\n \...,1091
...,...,...,...,...
1010,1010,29980814,\nName: ___ Unit No: _...,3329
1011,1011,29980930,\nName: ___ Unit No: ___...,1094
1012,1012,29987299,\nName: ___ Unit No: ___\...,2595
1013,1013,29988390,\nName: ___ Unit No: ...,3013


In [4]:
test_notes.drop(columns = ['Unnamed: 0'], inplace = True)

In [5]:
unique_hadm_notes = test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [184]:
unique_hadm_notes

,hadm_id,text
0,20019282,sex m orthopaedics allergies erythromycin base...
1,20022343,sex f obstetricsgynecology allergies bactrim p...
2,20025548,sex f obstetricsgynecology allergies no known ...
3,20027416,sex m otolaryngology allergies ragweed chief c...
4,20028418,sex f medicine allergies erythromycin base pac...
...,...,...
1010,29980814,sex f obstetricsgynecology allergies gelatin c...
1011,29980930,sex f medicine allergies morphine chief compla...
1012,29987299,sex m medicine allergies none attach laborato...
1013,29988390,sex f surgery allergies animal danderpollendus...


In [7]:
unique_hadm_notes = unique_hadm_notes.reset_index(drop = True)

In [8]:
unique_hadm_notes.text[0]

' \nName:  ___                 Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: ORTHOPAEDICS\n \nAllergies: \nerythromycin base\n \nAttending: ___.\n \nChief Complaint:\nLeft ankle fracture\n \nMajor Surgical or Invasive Procedure:\nORIF left ankle ___\n \nHistory of Present Illness:\n___ yr old man fall down 3 stairs while holding lumbar at \nconstruction\njob, resulting in immediate L ankle pain. ED ankle fracture, \nsurgery performed\n\n \nPast Medical History:\nPrevious R ankle fracture\nRLE cellulitis\nOsteoarthritis\n\n \nSocial History:\n___\nFamily History:\nN/A\n \nPhysical Exam:\nAOXO3\nsplint intact, positive CSM\nPain well controlled, no N/V\n\n \nPertinent Results:\n___ 08:28PM   WBC-9.5 RBC-4.49* HGB-13.7 HCT-41.0 MCV-91 \nMCH-30.5 MCHC-33.4 RDW-13.8 RDWSD-46.7*\n___ 08:28PM   GLUCOSE-96 UREA N-13 CREAT-0.7 SODIUM-145 \nPOTASSIUM-4.1 CHLORIDE-108 TOTAL CO2-24 ANION GAP-13\n___ 08:28PM   CA

In [9]:
def preprocess_clinical_notes(notes):
    # Convert the text to lowercase
    notes = notes.lower()
    
    # Remove special characters, irrelevant numbers, and newlines
    notes = re.sub(r'[^a-zA-Z0-9\s]', '', notes)
    notes = re.sub(r'\d+', '', notes)
    notes = re.sub(r'\s+', ' ', notes).strip()

    # Remove unnecessary sections
    notes = re.sub(r'(name|unit no|admission date|discharge date|date of birth|service|attending|major surgical or invasive procedure)', '', notes)

#     # Remove Chief Complaint
#     notes = re.sub(r'chief complaint', '', notes)

    # Remove History of Present Illness header and extra spaces
    notes = re.sub(r'history of present illness', '', notes)
    notes = re.sub(r' +', ' ', notes).strip()

    # Remove Past Medical History
    notes = re.sub(r'past medical history', '', notes)

    # Remove Social History
    notes = re.sub(r'social history', '', notes)

    # Remove Family History
    notes = re.sub(r'family history', '', notes)

#     # Remove Physical Exam
#     notes = re.sub(r'physical exam|admission exam|discharge exam', '', notes)

    # Remove Pertinent Results
    notes = re.sub(r'pertinent results|admission labs|relevant labs', '', notes)

#     # Remove Brief Hospital Course
#     notes = re.sub(r'brief hospital course', '', notes)

#     # Remove Acute Issues and Chronic Issues
#     notes = re.sub(r'acute issues|chronic issues', '', notes)

    # Remove Follow-up
    notes = re.sub(r'follow-up', '', notes)

    # Remove Medications on Admission
    notes = re.sub(r'medications on admission', '', notes)

    # Remove Discharge Medications
    notes = re.sub(r'discharge medications', '', notes)

    # Remove Discharge Disposition
    notes = re.sub(r'discharge disposition', '', notes)

#     # Remove Discharge Diagnosis
#     notes = re.sub(r'discharge diagnosis', '', notes)

#     # Remove Discharge Condition
#     notes = re.sub(r'discharge condition', '', notes)

    # Remove Discharge Instructions
    notes = re.sub(r'discharge instructions', '', notes)

    # Remove Transitional issues/outpatient plan
    notes = re.sub(r'transitional issuesoutpatient plan', '', notes)

    return notes


In [10]:
preprocess_clinical_notes(unique_hadm_notes.text[0])

'sex m orthopaedics allergies erythromycin base chief complaint left ankle fracture orif left ankle yr old man fall down stairs while holding lumbar at construction job resulting in immediate l ankle pain ed ankle fracture surgery performed  previous r ankle fracture rle cellulitis osteoarthritis   na physical exam aoxo splint intact positive csm pain well controlled no nv  pm wbc rbc hgb hct mcv mch mchc rdw rdwsd pm glucose urea n creat sodium potassium chloride total co anion gap pm calcium phosphate magnesium brief hospital course taken to operating room and underwent surgical fixation of ankle fracture on no complications post op doing well pain well controlled cleared by to go home without s  the preadmission medication list is accurate and complete multivitamins tab po daily naproxen mg po qhprn pain mild  acetaminophen mg po qh rx acetaminophen mg tablets by mouth every eight hours disp tablet refills aspirin mg po daily rx aspirin mg tablets by mouth once a day disp tablet ref

In [11]:
unique_hadm_notes['text'] = unique_hadm_notes['text'].apply(preprocess_clinical_notes)
unique_hadm_notes

,hadm_id,text
0,20019282,sex m orthopaedics allergies erythromycin base...
1,20022343,sex f obstetricsgynecology allergies bactrim p...
2,20025548,sex f obstetricsgynecology allergies no known ...
3,20027416,sex m otolaryngology allergies ragweed chief c...
4,20028418,sex f medicine allergies erythromycin base pac...
...,...,...
1010,29980814,sex f obstetricsgynecology allergies gelatin c...
1011,29980930,sex f medicine allergies morphine chief compla...
1012,29987299,sex m medicine allergies none attach laborato...
1013,29988390,sex f surgery allergies animal danderpollendus...


In [12]:
icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [13]:
i = 0
for index, row in unique_hadm_notes.iterrows():
    i +=1
    
    if i%100 ==0:
        print(f'{i} notes have been processed!', '\n')
        
        
    single_note = row['text']
    hadm_id = row['hadm_id']

    # Get the entities from the text
    all_entities = cat.get_entities(single_note)

    icd_codes = set()  # Using a set to automatically remove duplicates

    for entity_id, entity_data in all_entities['entities'].items():
        icd10 = entity_data.get('icd10', [])
        for code in icd10:
            if code:  # Check if the code is not an empty string
                icd_codes.add(code)

    icd10_data['hadm_id'].append(hadm_id)
    icd10_data['icd10_codes'].append(list(icd_codes))



100 notes have been processed! 

200 notes have been processed! 

300 notes have been processed! 

400 notes have been processed! 

500 notes have been processed! 

600 notes have been processed! 

700 notes have been processed! 

800 notes have been processed! 

900 notes have been processed! 

1000 notes have been processed! 



In [14]:
# Create a DataFrame from the processed data
df_icd10 = pd.DataFrame(icd10_data)
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M19.99, S82.80, W10, M79.67, L03.9, T81.9, R5..."
1,20022343,"[L53.9, R50.9, R10.4, O60.0, O32.1, R52.9, R00..."
2,20025548,"[R99, A09.9, R25.8, Z80.9, R19.8, K56.6, T88.7..."
3,20027416,"[C76.0, E78.5, R52.9, F41.9, C80.9, R22.1]"
4,20028418,"[D39.1, Z88.9, R11]"
...,...,...
1010,29980814,"[T14.0, R50.9, R79.8, A64, K59.0, R58, R19.8, ..."
1011,29980930,[Z88.9]
1012,29987299,"[R58, D61.9, R31, Z88.9, D47.1]"
1013,29988390,"[R50.9, R10.4, D72.8, E07.9, K80.5, N23, R19.8..."


In [ ]:
filtered_test_notes['icd_code'].unique()+

In [15]:
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [16]:
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(lambda lst: [reformat(x) for x in lst])
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M19., S82., W10., M79., L03., T81., R52., R11.]"
1,20022343,"[L53., R50., R10., O60., O32., R52., R00., R58..."
2,20025548,"[R99., A09., R25., Z80., R19., K56., T88., N94..."
3,20027416,"[C76., E78., R52., F41., C80., R22.]"
4,20028418,"[D39., Z88., R11.]"
...,...,...
1010,29980814,"[T14., R50., R79., A64., K59., R58., R19., E16..."
1011,29980930,[Z88.]
1012,29987299,"[R58., D61., R31., Z88., D47.]"
1013,29988390,"[R50., R10., D72., E07., K80., N23., R19., K81..."


In [60]:
# df_icd10.to_csv('test_set_mapped_icd_hadmids_1k.csv')

# unique_hadm_notes.to_csv('test_set_unique_notes_hadmids_1k.csv')


In [20]:
df_icd10.dtypes

hadm_id         int64
icd10_codes    object
dtype: object

In [200]:
def only_keep_unique_codes(input_list):
    set_list = set(input_list)
    return list(set_list)

In [201]:
df_icd10['icd10_codes'] = df_icd10['icd10_codes'].apply(only_keep_unique_codes)


In [202]:
#Make this on category level:
df_icd10

,hadm_id,icd10_codes
0,20019282,"[M19., L03., R11., T81., W10., S82., M79., R52.]"
1,20022343,"[R58., R42., N93., L53., R50., R10., R06., O32..."
2,20025548,"[T88., R58., R99., R51., R52., R19., K56., R11..."
3,20027416,"[C76., C80., F41., E78., R22., R52.]"
4,20028418,"[R11., Z88., D39.]"
...,...,...
1010,29980814,"[R58., A64., E16., F41., D61., R19., T14., R50..."
1011,29980930,[Z88.]
1012,29987299,"[R58., D61., R31., D47., Z88.]"
1013,29988390,"[N23., D72., K80., R19., K81., R50., R10., R11..."


In [204]:
df_icd10.drop_duplicates(subset = ['hadm_id'], inplace = True)

In [23]:
icd_codes_with_smallest_size_notes = pd.read_csv('icd_codes_for_1000_testing_notes_with_smallest_size.csv')
icd_codes_with_smallest_size_notes.drop(columns = ['Unnamed: 0'], inplace = True)
icd_codes_with_smallest_size_notes

,hadm_id,icd_code
0,20019282,S82.
1,20019282,Z72.
2,20019282,W10.
3,20019282,Y92.
4,20022343,O60.
...,...,...
8580,29990058,Z30.
8581,29990058,Z23.
8582,29990058,Z86.
8583,29990058,K21.


In [24]:
final_notes_with_icd_codes = pd.merge(unique_hadm_notes, icd_codes_with_smallest_size_notes, on='hadm_id', how='left')
final_notes_with_icd_codes

,hadm_id,text,icd_code
0,20019282,sex m orthopaedics allergies erythromycin base...,S82.
1,20019282,sex m orthopaedics allergies erythromycin base...,Z72.
2,20019282,sex m orthopaedics allergies erythromycin base...,W10.
3,20019282,sex m orthopaedics allergies erythromycin base...,Y92.
4,20022343,sex f obstetricsgynecology allergies bactrim p...,O60.
...,...,...,...
8580,29990058,sex f obstetricsgynecology allergies no known ...,Z30.
8581,29990058,sex f obstetricsgynecology allergies no known ...,Z23.
8582,29990058,sex f obstetricsgynecology allergies no known ...,Z86.
8583,29990058,sex f obstetricsgynecology allergies no known ...,K21.


In [206]:
predicted = []
for codes in df_icd10.icd10_codes:
    for code in codes:
        predicted.append(code)

In [87]:
unique_icd_codes = set(np.append(final_notes_with_icd_codes.icd_code.unique(), predicted)) #.union(set(icd_code for icd_list in df_icd10['icd10_codes'] for icd_code in icd_list))


In [208]:

unique_icd_codes = set(final_notes_with_icd_codes['icd_code']) #.union(set(icd_code for icd_list in df_icd10['icd10_codes'] for icd_code in icd_list))



In [209]:
len(unique_icd_codes)

769

In [210]:
final_notes_with_icd_codes.icd_code.nunique()

769

In [211]:
# Step 3: Initialize TP, FP, FN, TN counts
counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [212]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%100 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    counts[icd_code] = (len(TP), len(FP), len(FN), TN)

100 codes are done!
200 codes are done!
300 codes are done!
400 codes are done!
500 codes are done!
600 codes are done!
700 codes are done!


In [213]:
counts

{'F43.': (5, 2, 10, 0),
 'X99.': (1, 1, 0, 0),
 'W39.': (0, 0, 1, 0),
 'Z15.': (0, 0, 2, 0),
 'J70.': (0, 0, 1, 0),
 'D53.': (0, 0, 2, 0),
 'K66.': (2, 3, 1, 0),
 'K25.': (1, 5, 1, 0),
 'O11.': (0, 0, 7, 0),
 'S70.': (0, 1, 2, 0),
 'Z39.': (0, 0, 6, 0),
 'D65.': (0, 0, 6, 0),
 'Y04.': (0, 1, 2, 0),
 'F48.': (0, 0, 1, 0),
 'K60.': (1, 0, 0, 0),
 'S00.': (0, 0, 5, 0),
 'K62.': (1, 4, 8, 0),
 'J04.': (0, 0, 1, 0),
 'S31.': (0, 2, 2, 0),
 'K58.': (0, 0, 4, 0),
 'O09.': (0, 0, 36, 0),
 'R97.': (0, 0, 6, 0),
 'G46.': (0, 0, 1, 0),
 'R60.': (2, 54, 3, 0),
 'C34.': (5, 11, 5, 0),
 'N85.': (1, 1, 9, 0),
 'H46.': (0, 0, 1, 0),
 'Z30.': (0, 0, 4, 0),
 'C73.': (1, 5, 1, 0),
 'R62.': (0, 3, 1, 0),
 'E89.': (0, 0, 4, 0),
 'C01.': (0, 0, 1, 0),
 'H54.': (2, 1, 4, 0),
 'O71.': (0, 1, 8, 0),
 'R18.': (5, 5, 7, 0),
 'O67.': (0, 0, 1, 0),
 'J30.': (1, 8, 6, 0),
 'O62.': (0, 6, 12, 0),
 'Z63.': (0, 5, 1, 0),
 'W54.': (0, 0, 1, 0),
 'I05.': (0, 1, 1, 0),
 'E05.': (1, 2, 0, 0),
 'R58.': (1, 186, 0, 0),
 'S2

In [214]:



# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall


In [215]:
precisions

{'F43.': 0.7142857142857143,
 'X99.': 0.5,
 'W39.': 0,
 'Z15.': 0,
 'J70.': 0,
 'D53.': 0,
 'K66.': 0.4,
 'K25.': 0.16666666666666666,
 'O11.': 0,
 'S70.': 0.0,
 'Z39.': 0,
 'D65.': 0,
 'Y04.': 0.0,
 'F48.': 0,
 'K60.': 1.0,
 'S00.': 0,
 'K62.': 0.2,
 'J04.': 0,
 'S31.': 0.0,
 'K58.': 0,
 'O09.': 0,
 'R97.': 0,
 'G46.': 0,
 'R60.': 0.03571428571428571,
 'C34.': 0.3125,
 'N85.': 0.5,
 'H46.': 0,
 'Z30.': 0,
 'C73.': 0.16666666666666666,
 'R62.': 0.0,
 'E89.': 0,
 'C01.': 0,
 'H54.': 0.6666666666666666,
 'O71.': 0.0,
 'R18.': 0.5,
 'O67.': 0,
 'J30.': 0.1111111111111111,
 'O62.': 0.0,
 'Z63.': 0.0,
 'W54.': 0,
 'I05.': 0.0,
 'E05.': 0.3333333333333333,
 'R58.': 0.0053475935828877,
 'S26.': 0,
 'I67.': 0.0,
 'S68.': 0,
 'I73.': 0.38461538461538464,
 'T71.': 1.0,
 'Z09.': 0,
 'N92.': 0.0,
 'W10.': 1.0,
 'I69.': 0,
 'O76.': 0,
 'I21.': 0.3125,
 'Z83.': 0,
 'I31.': 0.5,
 'O9A.': 0,
 'C76.': 0.0,
 'E65.': 0.0,
 'N48.': 1.0,
 'H80.': 0,
 'S65.': 0,
 'J81.': 0.16666666666666666,
 'D55.': 0,
 'K

In [216]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(precisions.values()) / len(precisions)
macro_recall = sum(recalls.values()) / len(recalls)


In [217]:
macro_precision

0.25480627983822546

In [218]:
macro_recall

0.2614478634770317

In [219]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score



In [220]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(f1_scores.values()) / len(f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [221]:
macro_f1_score

0.21759007655623386

In [222]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall

# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in precisions.values() if precision > 0])

In [223]:
average_precision

0.5170079926005156

In [224]:
average_recall = np.mean([recall for recall in recalls.values() if recall > 0])

In [225]:
average_recall

0.5304839235193599

In [226]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [227]:
average_f1 = np.mean([f1_score for f1_score in f1_scores.values() if f1_score > 0])

In [228]:
average_f1

0.4414954323792708

In [229]:
len([f1_score for f1_score in f1_scores.values() if f1_score > 0])

379

#### Top 50 ICD codes identification

In [230]:
top_50_icd_codes = icd_codes_with_smallest_size_notes['icd_code'].value_counts().index[:50]
top_50_icd_codes

Index(['Y92.', 'O99.', 'I10.', 'E78.', 'Z87.', 'Z3A.', 'Z51.', 'Z85.', 'E11.',
       'Z79.', 'Z68.', 'F41.', 'K21.', 'F32.', 'Z66.', 'Z86.', 'E66.', 'I25.',
       'Z37.', 'E87.', 'J45.', 'I48.', 'E03.', 'F17.', 'G47.', 'Y83.', 'Z95.',
       'J96.', 'G93.', 'O26.', 'N17.', 'N18.', 'G89.', 'C78.', 'J44.', 'D64.',
       'R40.', 'O34.', 'Z80.', 'O09.', 'R11.', 'I50.', 'Z23.', 'S06.', 'M19.',
       'N40.', 'I12.', 'I95.', 'S62.', 'O70.'],
      dtype='object')

In [231]:
filtered_test_notes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'].isin(top_50_icd_codes)]

filtered_test_notes

,hadm_id,text,icd_code
3,20019282,sex m orthopaedics allergies erythromycin base...,Y92.
7,20022343,sex f obstetricsgynecology allergies bactrim p...,O34.
9,20022343,sex f obstetricsgynecology allergies bactrim p...,Z3A.
10,20022343,sex f obstetricsgynecology allergies bactrim p...,Z37.
11,20025548,sex f obstetricsgynecology allergies no known ...,O99.
...,...,...,...
8578,29990058,sex f obstetricsgynecology allergies no known ...,Z3A.
8579,29990058,sex f obstetricsgynecology allergies no known ...,Z37.
8581,29990058,sex f obstetricsgynecology allergies no known ...,Z23.
8582,29990058,sex f obstetricsgynecology allergies no known ...,Z86.


In [232]:
filtered_unique_hadm_notes = filtered_test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')


In [233]:
filtered_unique_hadm_notes = filtered_unique_hadm_notes.reset_index(drop = True)
filtered_unique_hadm_notes

,hadm_id,text
0,20019282,sex m orthopaedics allergies erythromycin base...
1,20022343,sex f obstetricsgynecology allergies bactrim p...
2,20025548,sex f obstetricsgynecology allergies no known ...
3,20027416,sex m otolaryngology allergies ragweed chief c...
4,20028418,sex f medicine allergies erythromycin base pac...
...,...,...
929,29980814,sex f obstetricsgynecology allergies gelatin c...
930,29980930,sex f medicine allergies morphine chief compla...
931,29987299,sex m medicine allergies none attach laborato...
932,29988390,sex f surgery allergies animal danderpollendus...


In [234]:
filtered_icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}



In [235]:
unique_icd_codes = set(filtered_test_notes['icd_code'])

In [236]:
# Step 3: Initialize TP, FP, FN, TN counts
filtered_counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [237]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%10 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = filtered_test_notes[filtered_test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = df_icd10[df_icd10['icd10_codes'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    filtered_counts[icd_code] = (len(TP), len(FP), len(FN), TN)

10 codes are done!
20 codes are done!
30 codes are done!
40 codes are done!
50 codes are done!


In [238]:
filtered_counts

{'O26.': (0, 0, 49, 0),
 'S62.': (3, 1, 14, 0),
 'Z95.': (0, 1, 48, 0),
 'Z87.': (2, 14, 184, 0),
 'Z37.': (0, 0, 86, 0),
 'O34.': (0, 0, 42, 0),
 'E78.': (70, 14, 127, 0),
 'Y83.': (7, 0, 64, 0),
 'J44.': (25, 7, 20, 0),
 'F17.': (0, 4, 73, 0),
 'I25.': (17, 5, 50, 0),
 'Z79.': (0, 0, 122, 0),
 'D64.': (13, 20, 32, 0),
 'O70.': (0, 1, 33, 0),
 'J45.': (41, 29, 39, 0),
 'C78.': (7, 3, 26, 0),
 'F32.': (28, 32, 72, 0),
 'I50.': (15, 5, 24, 0),
 'I12.': (0, 2, 34, 0),
 'Z86.': (1, 20, 81, 0),
 'G47.': (18, 25, 53, 0),
 'I10.': (8, 27, 216, 0),
 'N17.': (9, 1, 44, 0),
 'S06.': (8, 6, 19, 0),
 'K21.': (42, 11, 62, 0),
 'I95.': (12, 13, 22, 0),
 'G89.': (0, 0, 51, 0),
 'F41.': (48, 48, 58, 0),
 'O99.': (2, 0, 128, 0),
 'N40.': (1, 0, 33, 0),
 'Z85.': (0, 2, 111, 0),
 'Z23.': (0, 0, 40, 0),
 'I48.': (56, 3, 18, 0),
 'E03.': (50, 6, 24, 0),
 'R11.': (22, 144, 19, 0),
 'O09.': (0, 0, 36, 0),
 'Z3A.': (0, 0, 193, 0),
 'R40.': (8, 53, 12, 0),
 'N18.': (15, 3, 37, 0),
 'Z51.': (0, 0, 133, 0),
 'M

In [239]:

# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall

In [240]:
filtered_precisions

{'O26.': 0,
 'S62.': 0.75,
 'Z95.': 0.0,
 'Z87.': 0.125,
 'Z37.': 0,
 'O34.': 0,
 'E78.': 0.8333333333333334,
 'Y83.': 1.0,
 'J44.': 0.78125,
 'F17.': 0.0,
 'I25.': 0.7727272727272727,
 'Z79.': 0,
 'D64.': 0.3939393939393939,
 'O70.': 0.0,
 'J45.': 0.5857142857142857,
 'C78.': 0.7,
 'F32.': 0.4666666666666667,
 'I50.': 0.75,
 'I12.': 0.0,
 'Z86.': 0.047619047619047616,
 'G47.': 0.4186046511627907,
 'I10.': 0.22857142857142856,
 'N17.': 0.9,
 'S06.': 0.5714285714285714,
 'K21.': 0.7924528301886793,
 'I95.': 0.48,
 'G89.': 0,
 'F41.': 0.5,
 'O99.': 1.0,
 'N40.': 1.0,
 'Z85.': 0.0,
 'Z23.': 0,
 'I48.': 0.9491525423728814,
 'E03.': 0.8928571428571429,
 'R11.': 0.13253012048192772,
 'O09.': 0,
 'Z3A.': 0,
 'R40.': 0.13114754098360656,
 'N18.': 0.8333333333333334,
 'Z51.': 0,
 'M19.': 0.3333333333333333,
 'G93.': 0.6129032258064516,
 'E66.': 0.6379310344827587,
 'Y92.': 0,
 'E87.': 0.875,
 'Z68.': 0,
 'E11.': 0.7368421052631579,
 'Z80.': 0.043478260869565216,
 'J96.': 0.6538461538461539,
 'Z

In [241]:
filtered_recalls

{'O26.': 0.0,
 'S62.': 0.17647058823529413,
 'Z95.': 0.0,
 'Z87.': 0.010752688172043012,
 'Z37.': 0.0,
 'O34.': 0.0,
 'E78.': 0.3553299492385787,
 'Y83.': 0.09859154929577464,
 'J44.': 0.5555555555555556,
 'F17.': 0.0,
 'I25.': 0.2537313432835821,
 'Z79.': 0.0,
 'D64.': 0.28888888888888886,
 'O70.': 0.0,
 'J45.': 0.5125,
 'C78.': 0.21212121212121213,
 'F32.': 0.28,
 'I50.': 0.38461538461538464,
 'I12.': 0.0,
 'Z86.': 0.012195121951219513,
 'G47.': 0.2535211267605634,
 'I10.': 0.03571428571428571,
 'N17.': 0.16981132075471697,
 'S06.': 0.2962962962962963,
 'K21.': 0.40384615384615385,
 'I95.': 0.35294117647058826,
 'G89.': 0.0,
 'F41.': 0.4528301886792453,
 'O99.': 0.015384615384615385,
 'N40.': 0.029411764705882353,
 'Z85.': 0.0,
 'Z23.': 0.0,
 'I48.': 0.7567567567567568,
 'E03.': 0.6756756756756757,
 'R11.': 0.5365853658536586,
 'O09.': 0.0,
 'Z3A.': 0.0,
 'R40.': 0.4,
 'N18.': 0.28846153846153844,
 'Z51.': 0.0,
 'M19.': 0.16129032258064516,
 'G93.': 0.4523809523809524,
 'E66.': 0.381

In [242]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(filtered_precisions.values()) / len(filtered_precisions)
macro_recall = sum(filtered_recalls.values()) / len(filtered_recalls)


In [243]:
macro_precision

0.3985932454996356

In [244]:
macro_recall

0.19260102345192462

In [245]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score



In [246]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(filtered_f1_scores.values()) / len(filtered_f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation.

In [247]:
macro_f1_score

0.23011675504957096

In [248]:
# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall    
    
    
    
    
    
# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in filtered_precisions.values() if precision > 0])

In [249]:
average_precision

0.6039291598479328

In [250]:
average_recall = np.mean([recall for recall in filtered_recalls.values() if recall > 0])

In [251]:
average_recall

0.29181973250291615

In [252]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [253]:
average_f1 = np.mean([f1_score for f1_score in filtered_f1_scores.values() if f1_score > 0])

In [254]:
average_f1

0.34866175007510763